## 【 D0120_work_이준기 】
### - mnist 손글씨 다중클래스 분류

다중 분류 시
1. softmax를 직접 사용
2. CrossEntropyLoss로 손실 계산 (softmax 자동 적용)

-> 두 방법 중 수업 시간에 배운 softmax를 직접 적용해서 학습시키는 방법을 사용해보았습니다.

In [41]:
## [1-1] 모듈 로딩
import pandas as pd                 # 데이터 분석 및 처리용 모듈
import torch                        # 텐서 및 수치, 기본 함수용 모듈
import torch.nn as nn               # 인공신경망 관련 모듈
import torch.nn.functional as F     # 인공신경망 함수(AF, LF, MF) 관련 모듈
import torch.optim as optim         # 경사하강법 알고리즘으로 최적화 관련 모듈

from torchinfo import summary       # 모델 구조 확인용 유틸 모듈

In [42]:
## [1-2] 데이터 관련
DATA_FILE = '../Data/mnist_train.csv'

##- 데이터 로딩
dataDF = pd.read_csv(DATA_FILE)
dataDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 785 entries, 5 to 0.617
dtypes: int64(785)
memory usage: 59.9 MB


In [43]:
## [1-3] 데이터 -> Tensor 변환
featureDF = dataDF.iloc[:, 1:]
xTS       = torch.tensor(featureDF.values, dtype=torch.float32)

targetDF = dataDF.iloc[:, 0]
yTS      = torch.tensor(targetDF.values)

print(f" xTS : {xTS.shape},  yTS : {yTS.shape}")

 xTS : torch.Size([10000, 784]),  yTS : torch.Size([10000])


In [ ]:
## -------------------------------------------------------------
##          입력수         퍼셉트론수/출력수         AF
## -------------------------------------------------------------
## 입력층       784개             784개           ★ Pytorch에는
##                                               입력층 클래스 X
##                                               입력 텐서를 입력층으로 간주
## 은닉층      784개              128개           ReLU
## 은닉층      128개               64개           ReLU
## 출력층      64개                10개           -     분류
## -------------------------------------------------------------
## 클래스이름 : MNISTModel
## 부모클래스 : nn.Module
## 오버라이딩 : __init__(self)   : 층 구성 요소 인스턴스 생성
##            forward(self, x) : 순전파 진행 메서드
##                               x -> 입력층으로 간주!
## -------------------------------------------------------------
class MNISTModel(nn.Module):
    ##- 층 구성 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()
        self.hd1_layer=nn.Linear(784, 128)
        self.hd2_layer=nn.Linear(128, 64)
        self.out_layer=nn.Linear(64, 10)

    ##- 순전파 진행 메서드
    def forward(self, x):
        ## 입력층 -> 은닉층 :  784 -> 128
        out = self.hd1_layer(x)
        out = F.relu(out)

        ## 은닉층 -> 은닉층 :  128 -> 64
        out = self.hd2_layer(out)
        out = F.relu(out)

        ## 은닉층 -> 은닉층 :  64 -> 10
        out = self.out_layer(out)

        out = F.log_softmax(out, dim=1) # 다중 뷴류 모델이므로 softmax 활성화 함수 사용

        return out

In [ ]:
model = MNISTModel()
summary(model, input_size=(1, 784))

Layer (type:depth-idx)                   Output Shape              Param #
MNISTModel                               [1, 10]                   --
├─Linear: 1-1                            [1, 128]                  100,480
├─Linear: 1-2                            [1, 64]                   8,256
├─Linear: 1-3                            [1, 10]                   650
Total params: 109,386
Trainable params: 109,386
Non-trainable params: 0
Total mult-adds (M): 0.11
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.44
Estimated Total Size (MB): 0.44

In [46]:
EPOCHS     = 1000                           ## 처음 ~ 끝까지 학습 횟수
BATCH_SIZE = 200                            ## 1번 학습할 데이터 크기
COUNT      = featureDF.shape[0]/BATCH_SIZE  ## 1에포크에 파라미터 업데이트 횟수
COUNT

50.0

In [ ]:
## [3-2] 학습 관련 인스턴스들
## -> 모델 인스턴스
model = ScoreModel()

## -> 손실 계산 인스턴스
loss_fn = nn.NLLLoss()      # softmax 사용 시 사용하는 손실 계산 인스턴스

## -> 최적화 인스턴스
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [49]:
## 에포크 당 loss 저장 변수
history = []

## 학습 진행
for epoch in range(EPOCHS):
    total_loss = 0
    for idx in range(int(COUNT)):
        ##- 배치크기만큼 데이터 추출 인덱스
        sIdx = idx  * BATCH_SIZE
        eIdx = sIdx + BATCH_SIZE

        ##- 배치크기만큼 순전파 진행 ==> 예측값 추출+
        pre_y = model(xTS[sIdx:eIdx])

        ##- 손실계산
        loss = loss_fn(pre_y, yTS[sIdx:eIdx])
        
        ##- 역전파 + 최적화
        optimizer.zero_grad()
        loss.backward()     ##<- 역전파: 경사하강법으로 W,b 계산진행
        optimizer.step()      ##<- 새로운 W,b 업데이트 진행

        ##- 배치 크기 loss 누적
        total_loss += loss.item()
        

    ## 50에포크마다 손실 출력
    if not epoch%50:
        print(f'[{epoch+1:03}_에포크] loss : {total_loss/COUNT:.5f}')

[001_에포크] loss : 0.00000
[051_에포크] loss : 0.00000
[101_에포크] loss : 0.00000
[151_에포크] loss : 0.00000
[201_에포크] loss : 0.00000
[251_에포크] loss : 0.00000
[301_에포크] loss : 0.00000
[351_에포크] loss : 0.00006
[401_에포크] loss : 0.00001
[451_에포크] loss : 0.00000
[501_에포크] loss : 0.00000
[551_에포크] loss : 0.00000
[601_에포크] loss : 0.00000
[651_에포크] loss : 0.00000
[701_에포크] loss : 0.00000
[751_에포크] loss : 0.04865
[801_에포크] loss : 0.00008
[851_에포크] loss : 0.00001
[901_에포크] loss : 0.00000
[951_에포크] loss : 0.00000
